In [1]:
import numpy as np
import scipy as sp

In [25]:
data_path = "../Data/iPad_s8/iPad_s8_train_data_3.txt"

label_arr = []
feature_arr = []

with open(data_path, 'r') as data_file:
        for line in data_file:
            value = line.split('|')
            
            label_line = value[1]
            label_rgb = label_line.split(' ')
            label_arr.append([int(label_rgb[1]), int(label_rgb[2]), int(label_rgb[3])])
            
            feature_line = value[2]
            feature_rgb = feature_line.split(' ')
            feature_arr.append([1, int(feature_rgb[1]), int(feature_rgb[2]), int(feature_rgb[3].replace('\n', ''))])
            
#print(label_arr[:10])
print(len(label_arr))

#print(feature_arr[:10])
print(len(feature_arr))

11847024
11847024


In [26]:
x = np.array(feature_arr) # target value => S8
#print("x : ", x[:10])
xt = x.transpose()
#print("xt : ", xt[:10])

y = np.array(label_arr) # original value => iPad Mini 4
#print("y : ", y[:10])

In [27]:
# Find w = (XtX)^-1 Xt Y
prefix = np.matmul(xt, x)
prefix_inv = np.linalg.pinv(prefix)
w = np.matmul(prefix_inv, np.matmul(xt, y))
print(w)

[[ -4.19177110e+00  -1.88891061e+00  -2.94830842e-02]
 [  9.09799628e-01   5.31777804e-02  -1.35308044e-02]
 [ -5.29412699e-02   8.65753128e-01   1.88806811e-02]
 [ -1.14148538e-03  -5.45939643e-02   7.91649258e-01]]


In [28]:
np.matmul(x, w)

array([[  30.40484047,   91.21372668,  142.46535772],
       [  48.62137087,  142.96221838,   75.86463836],
       [ 108.0045137 ,  134.4077601 ,  161.14274047],
       ..., 
       [  42.8812287 ,   82.17112787,  132.56309799],
       [  43.85875762,  142.67307857,  113.98809883],
       [ 124.27045853,  136.87809054,  164.10354439]])

In [79]:
from PIL import Image

image_path = "./example_image/Snack(S8+)_resize.jpg"
img = Image.open(image_path)
pixel_list = []
for i in range(img.size[0]):    
        for j in range(img.size[1]):
            pixel_list.append(img.getpixel((i,j)))
            #print("R : ", img.getpixel((i,j))[0])

#print(pixel_list[:10])
#print(len(pixel_list))

pixel_set = set(pixel_list)
original_value = list(pixel_set)
#print(original_value[:10])
#print(len(original_value))

original_unique_value = []
for point in original_value:
    original_unique_value.append([1, point[0], point[1], point[2]])

original_value_arr = np.array(original_unique_value)
#print(original_value_arr[:10])


In [80]:
predict_arr = np.matmul(original_value_arr, w)
print(predict_arr)
print(len(predict_arr))

[[ 177.00281826  162.73544461   85.30956398]
 [ 200.39840995  189.80818258   14.94058461]
 [  57.49419765   53.70613661  115.87985772]
 ..., 
 [ 142.19265525  130.93110029  126.43125647]
 [  98.8096607    85.62525618   53.25115826]
 [ 166.95673846   85.40985446  113.19334083]]
194984


In [81]:
combined_dict = {}
i = 0
for e in original_value:
    combined_dict[e] = predict_arr[i]
    i += 1

i = 0
for key in combined_dict:
    print(key , " => ", combined_dict[key])
    i+=1
    if i > 5:
        break

(210, 184, 107)  =>  [ 177.00281826  162.73544461   85.30956398]
(237, 208, 18)  =>  [ 200.39840995  189.80818258   14.94058461]
(72, 69, 146)  =>  [  57.49419765   53.70613661  115.87985772]
(193, 160, 55)  =>  [ 162.86617229  143.89223342   43.92068987]
(190, 169, 152)  =>  [ 159.5495779   146.22886369  120.92118647]
(229, 223, 197)  =>  [ 192.12156799  192.59673764  157.03725851]


In [82]:
def correct_range(pixel_value):
    if pixel_value < 0:
        pixel_value = 0
        
    if pixel_value > 255:
        pixel_value = 255
        
    return pixel_value

img = Image.open(image_path)
pixels = img.load()
for i in range(img.size[0]):    
        for j in range(img.size[1]):
            key = img.getpixel((i,j))
            R = int(combined_dict[key][0])
            G = int(combined_dict[key][1])
            B = int(combined_dict[key][2])
            
            R = correct_range(R)
            G = correct_range(G)
            B = correct_range(B)
            
            pixels[i,j] = (R, G, B)
            
img.save("./example_image/iPad_s8_augmented_img/Snack(S8+)_resize_Nannnnn.jpg")
print("Finish Write Image")

Finish Write Image
